## Requirements

In [ ]:
! git clone https://github.com/lopezjuanma96/vits.git
%cd vits
%pip install -r requirements.txt
# build monotonic align
%cd monotonic_align/
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..
!mkdir data

## Get Data

### download

In [ ]:
!wget https://huggingface.co/datasets/lopezjm96/spanish_voices/resolve/main/spanish_voices.zip
!unzip spanish_voices.zip

### to filelists

In [ ]:
import random

with open("spanish_voices/transcript_coqui.txt", encoding="utf-8") as f:
  original = f.read().split('\n')

ratio = 0.1
def split_data(data, ratio):
  train_index = list(range(len(data)))
  val_index = []
  while len(val_index) < ratio * len(train_index):
    val_index.append(train_index.pop(random.randint(0, len(train_index)-1)))
  return [data[i] for i in train_index], [data[i] for i in val_index]

train, val = split_data(original, ratio)

with open("filelists/spanish_voices_train.txt", "w", encoding="utf-8") as f:
  f.write('\n'.join(train))

with open("filelists/spanish_voices_val.txt", "w", encoding="utf-8") as f:
  f.write('\n'.join(val))

## Preprocess

In [ ]:
!python preprocess.py --text_index 1 --filelists filelists/spanish_voices_train.txt filelists/spanish_voices_val.txt --text_cleaners spanish_cleaners 

## Train

In [ ]:
!python train.py --config configs/es_base.json